## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [35]:
!pip install line_profiler
%load_ext line_profiler


1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [13]:
import random
import math

In [16]:
N=1000000
A = [random.randint(0, 1000) for _ in range(N)]
B = [a + 100 for a in A]
res=sum(B) / len(B)
res

TypeError: sum() takes 0 positional arguments but 1 was given

In [17]:
import numpy as np
import statistics
A = np.random.randint(0,1000, size=(1000000))
B=A+100
statistics.mean(B)


599

In [18]:
N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = []
for a in A:
    B.append(a + 100)
mean_B = sum(B)/len(B)
mean_B

TypeError: sum() takes 0 positional arguments but 1 was given

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [47]:
import random
import string
import pandas as pd

N = 2000000
df = pd.DataFrame({
    'col1': [random.randint(0, 1000) for _ in range(N)],
    'col2': [random.randint(0, 1000) for _ in range(N)],
    'col3': [random.randint(0, 1000) for _ in range(N)],
    'col4': [random.randint(0, 1000) for _ in range(N)],
})

df['key'] = ''.join(random.choice(string.ascii_letters) for _ in range(N))




In [40]:
import pandas as pd
import random
import string

N = 2000000
data = {'col1': [random.randint(0, 100) for i in range(N)],
        'col2': [random.uniform(0, 1) for i in range(N)],
        'col3': [random.choice([True, False]) for i in range(N)],
        'col4': [random.choice(['A', 'B', 'C', 'D']) for i in range(N)]}

df = pd.DataFrame(data)

keys = ''.join(random.choice(string.ascii_uppercase) for _ in range(N))
df['key'] = list(keys)

subset = df[df['key'].str[:5].isin(list(string.ascii_uppercase)[:5])]

print(subset)

         col1      col2   col3 col4 key
3          43  0.082364  False    B   C
7          77  0.248854  False    D   B
12         86  0.419805  False    D   C
21         63  0.015310   True    A   B
51         68  0.159927   True    A   B
...       ...       ...    ...  ...  ..
1999975    10  0.317377  False    D   E
1999983    64  0.736150   True    D   B
1999985    60  0.940648  False    C   C
1999988     2  0.105155  False    D   B
1999993    12  0.172362   True    C   B

[383464 rows x 5 columns]


In [41]:
import numba



def g3(df): # ERROR
    res = pd.DataFrame ()
    for letter in ['a', 'b', 'c', 'd', 'e']:
        res = pd.concat([res, df[df['key']==letter]], axis=0)
    return res

g3(df)

,col1,col2,col3,col4,key


## Лабораторная работа 3

In [34]:
!pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [42]:
import pandas as pd
import time
import numpy as np 
from numba import jit,njit


In [43]:
recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

In [44]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

In [45]:
# A. С использованием метода DataFrame.iterrows исходной таблицы:

def mean_rating_2010a(reviews):
    total = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    return total/count
# mean_rating_2010a(reviews)
%time

CPU times: total: 0 ns
Wall time: 0 ns


In [46]:
# Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год:

def mean_rating_2010b(reviews_2010):
    total = 0
    count = 0
    for index, row in reviews_2010.iterrows():
        total += row['rating']
        count += 1
    return total/count
# mean_rating_2010b(reviews)
%time

CPU times: total: 0 ns
Wall time: 0 ns


In [47]:
# В. С использованием метода Series.mean:

def mean_rating_2010c(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

mean_rating_2010c(reviews)
%time

CPU times: total: 0 ns
Wall time: 0 ns


In [48]:
# Проверим, что результаты работы всех написанных функций корректны и совпадают:

reviews_2010 = reviews[reviews['date'].dt.year == 2010]

assert mean_rating_2010a(reviews) == mean_rating_2010b(reviews_2010) == mean_rating_2010c(reviews)

In [49]:
import pandas as pd

def function_1B(df: pd.DataFrame) -> float:
    # Группируем данные по столбцу 'A'
    grouped_data = df.groupby('A')
    
    # Вычисляем среднее значение для каждой группы
    mean_values = grouped_data['B'].mean()
    
    # Вычисляем сумму средних значений
    result = mean_values.sum()
    
    return result

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [50]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%%time

UsageError: Line magic function `%%time` not found.


In [ ]:
from collections import defaultdict
from multiprocessing import Pool
from numba import jit

@jit(nopython=True)
def count_words(text, words):
    stats = defaultdict(int)
    for word in text.split():
        if word in words:
            stats[word] += 1
    return stats

def analyze_reviews(reviews, words):
    stats = defaultdict(int)
    with Pool() as p:
        results = p.starmap(count_words, [(review, words) for review in reviews])
        for result in results:
            for word, count in result.items():
                stats[word] += count
    return stats


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
import pandas as pd

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += abs(actual_rating - rating) / rating
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [ ]:
# С использованием векторизованных операций и методов массивов numpy, но без использования numba:

import pandas as pd
import numpy as np

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += np.abs(actual_rating - rating) / rating
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [ ]:
# С использованием numba и векторизованных операций и методов массивов numpy:

import pandas as pd
import numpy as np
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    return np.abs(actual_rating - rating) / rating

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [51]:
# Без использования векторизованных операций и методов массивов numpy, но с использованием numba
import pandas as pd
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    mape_sum = 0
    for i in range(len(actual_rating)):
        mape_sum += abs(actual_rating[i] - rating) / rating
    return mape_sum

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [52]:
# Без использования векторизованных операций и методов массивов numpy и без использования numba
import pandas as pd

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += abs(actual_rating - rating) / rating
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result